In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [3]:
# Load dataset
file_path = '../data/Singtel_stock_data.csv'
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df.set_index('Date', inplace=True)

# Prepare the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = df['Adj Close'].values.reshape(-1, 1)
scaled_data = scaler.fit_transform(data)

# Create a function to prepare the dataset for LSTM
def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i - time_step:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

# Prepare training data
time_step = 60
X, Y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and validation sets
train_size = int(len(X) * 0.8)
X_train, X_valid = X[:train_size], X[train_size:]
Y_train, Y_valid = Y[:train_size], Y[train_size:]

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, Y_train, batch_size=1, epochs=5)

# Validate the model
predictions = model.predict(X_valid)

# Reverse scaling to get the original values
predictions = scaler.inverse_transform(predictions)
Y_valid = scaler.inverse_transform(Y_valid.reshape(-1, 1))

# Plot the predictions using plotly.express
validation_df = pd.DataFrame({
    'Date': df.index[train_size + time_step:],
    'Actual Price': Y_valid.flatten(),
    'Predicted Price': predictions.flatten()
})
fig = px.line(validation_df, x='Date', y=['Actual Price', 'Predicted Price'], title='Actual vs Predicted Stock Prices')
fig.update_layout(xaxis_title='Date', yaxis_title='Close Price')
fig.show()

# Prepare the data for prediction for the next 360 days
last_60_days = scaled_data[-time_step:]
future_predictions = []

for _ in range(360):
    input_data = last_60_days.reshape(1, time_step, 1)
    predicted_value = model.predict(input_data)
    future_predictions.append(predicted_value[0, 0])
    last_60_days = np.append(last_60_days[1:], predicted_value[0, 0])
    last_60_days = last_60_days.reshape(-1, 1)

# Inverse transform predictions
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Create future dates starting from the last date in the dataset
last_date = pd.to_datetime(df.index[-1])
future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 361)]

# Combine historical and future predictions
future_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted Price': future_predictions.flatten()
})
combined_df = pd.concat([validation_df[['Date', 'Actual Price', 'Predicted Price']], future_df], ignore_index=True)

# Plot combined predictions using plotly.express
fig = px.line(combined_df, x='Date', y=['Actual Price', 'Predicted Price'], title='Actual and Predicted Stock Prices with Future Forecast')
fig.update_layout(xaxis_title='Date', yaxis_title='Close Price')
fig.show()


c:\Users\LJM\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
928/928 ━━━━━━━━━━━━━━━━━━━━ 3416s 4s/step - loss: 0.0035
Epoch 2/5
928/928 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - loss: 7.8037e-04
Epoch 3/5
928/928 ━━━━━━━━━━━━━━━━━━━━ 34s 36ms/step - loss: 6.8372e-04
Epoch 4/5
928/928 ━━━━━━━━━━━━━━━━━━━━ 43s 47ms/step - loss: 6.1860e-04
Epoch 5/5
928/928 ━━━━━━━━━━━━━━━━━━━━ 50s 54ms/step - loss: 4.9092e-04
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [5]:
# Plot future predictions using plotly.express
future_df = pd.DataFrame({
    'Day': range(1, 361),
    'Predicted Price': future_predictions.flatten()
})
fig = px.line(future_df, x='Day', y='Predicted Price', title='Next 90 Days Stock Price Forecast')
fig.update_layout(xaxis_title='Days', yaxis_title='Close Price')
fig.show()